# Static Simulation for the Reputation Score

Reference: https://hackmd.io/8CSLDfKNSAyyl-OfEzOdDQ

In [21]:
import scipy.stats as st
from dataclasses import dataclass
import seaborn as sns
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata

In [22]:
Filecoin = float
Days = float


@dataclass
class Deal():
    payment: Filecoin
    duration: Days
    collateral: Filecoin
    finished: bool
    slashed: bool

    def deal_payoff(self,
                    slash_penalty: float) -> float:
        S = slash_penalty if self.slashed else 1.0
        return (self.duration * self.collateral * self.finished * S / self.payment)


@dataclass
class Provider():
    deals: list[Deal]

    def reputation_payoff(self,
                          slash_penalty: float,
                          alpha: float) -> float:
                          
        return sum(d.deal_payoff(slash_penalty) ** alpha
                   for d
                   in self.deals)


In [23]:
SLASH_PENALTY = -10
ALPHA = 1/2
N_PROVIDERS = 1_000
DEALS_PER_PROVIDER_MU = 5

P_DEAL_FINISHED = 0.5
P_DEAL_SLASHED = 0.01

providers = []
for _ in range(N_PROVIDERS):
    N_provider_deals = st.poisson(mu=DEALS_PER_PROVIDER_MU).rvs()
    durations = st.gamma.rvs(a=2, scale=20, size=N_provider_deals)
    collaterals = st.gamma.rvs(a=1, scale=1, size=N_provider_deals)
    payments = collaterals / 10
    finished = st.bernoulli.rvs(p=P_DEAL_FINISHED, size=N_provider_deals)
    slashed = st.bernoulli.rvs(p=P_DEAL_SLASHED, size=N_provider_deals)

    deals_data = zip(payments, durations, collaterals, finished, slashed)

    deals = [Deal(*deal_data)
             for deal_data
             in deals_data]

    providers.append(Provider(deals))
provider_reputation_payoffs = [p.reputation_payoff(slash_penalty=SLASH_PENALTY, alpha=ALPHA)
                               for p 
                               in providers]

/var/folders/y4/dvc6qr5j30s3frwztjt5shg40000gn/T/ipykernel_10353/1179977817.py:27: RuntimeWarning:

invalid value encountered in double_scalars



In [24]:
fig = px.histogram(provider_reputation_payoffs)
fig.show()

In [25]:
provider_ranking = rankdata(provider_reputation_payoffs)
provider_norm_ranking = 100 * provider_ranking / max(provider_ranking)

fig = px.scatter(x=provider_norm_ranking,
           y=provider_reputation_payoffs,
           labels={'x': 'Provider Reputation', 'y': 'Provider Reputation Payoff'})
fig.show()
